In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import time
import json
import csv
import os
import pandas as pd
from tqdm import tqdm_notebook
from urllib.error import URLError
import time

In [2]:
f = open("./scraped-data/UE-categories-all.txt")
lines = f.read().split('\n')
categories = set(lines)
if '' in categories:
    categories.remove('')
len(categories)

89

In [3]:
def get_city_rests(city_url, categories, debug=False):
    start_time = time.time()
    rests = set()
    categ_stats = {}
    cityname = city_url.split('/')[-1]
    try:
        req = Request(city_url, headers={'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'})
        webpage = urlopen(req, timeout=20).read()
        soup = BeautifulSoup(webpage, 'html.parser')
    except Exception as e:
        print(f"Error: {e}, url: {city_url}")
        return rests, categ_stats
    
    for category in tqdm_notebook(categories, desc=cityname):
        category = category.lower().replace(" ", "-")
        category_city_url = "https://www.ubereats.com/category/" + cityname + "/" + category
        try:
            req = Request(category_city_url, headers={'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'})

            webpage = urlopen(req, timeout=20).read()
            soup = BeautifulSoup(webpage, 'html.parser')
            if debug == True:
                print(soup.title)
            rest_links = soup.find_all("h3", class_="b7 ch b9 b4 f0 al aj f1 ba")
            categ_stats[category] = len(rest_links)
            old_rests_size = len(rests)

            if debug == True:
                print(f"found: {len(rest_links)}, old size: {old_rests_size}")
            for link in rest_links:
                rests.add(link.a['href'])
            if debug == True:
                print(f"added: {len(rests)-old_rests_size}, new size: {len(rests)}")
        except Exception as e:
            if debug == True:
                print(f"\nerror: {e}\n url: {category_city_url}")
            categ_stats[category] = 0
    elapsed_time = time.time()-start_time
    if debug == True:
        print(f"time elapsed: {elapsed_time}")
    return rests, categ_stats

In [14]:
def store_urls(city_url, city_rests):
    cityname = city_url.split('/')[-1]
    filename = './scraped-data/rest-urls/2100-2200/'+cityname+'.csv'
    city_df = pd.DataFrame(columns=['url', 'city'])
    
    i=0
    for rest_url in city_rests:
        city_df.loc[i] = [
            "https://www.ubereats.com" + rest_url,
            cityname
        ]
        i = i+1
    city_df.to_csv(filename, index=False)

In [15]:
cities = pd.read_csv('./scraped-data/UE-cities.csv')['url']
len(cities)

3294

In [17]:
for city_url in tqdm_notebook(cities[2100:2200], desc="city-rest-urls"):
    city_rests, categ_stats = get_city_rests(city_url, categories, debug=False)
    store_urls(city_url, city_rests)

Error: HTTP Error 404: Not Found, url: https://www.ubereats.com/location/woodbury-new-jersey
Error: HTTP Error 404: Not Found, url: https://www.ubereats.com/location/woodland-park-new-jersey
Error: HTTP Error 404: Not Found, url: https://www.ubereats.com/location/rio-rancho-new-mexico
Error: HTTP Error 404: Not Found, url: https://www.ubereats.com/location/santa-fe
Error: HTTP Error 404: Not Found, url: https://www.ubereats.com/location/airmont-new-york
Error: HTTP Error 404: Not Found, url: https://www.ubereats.com/location/bayport-new-york


KeyboardInterrupt: 

In [13]:
cities[1731]

'https://www.ubereats.com/location/creve-coeur-missouri'